# 导入模块

In [9]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 连接数据库

In [10]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 查询数据

In [11]:
sql = '''
select 
om.create_time, om.order_number, tprm.order_id, tmu.true_name 客户姓名, tol.delivery_province_name, tol.delivery_city_name, cc.name as channel_name, ymos.refund_date, ymos.reality_refund_date
,tmu.id_card_num, om.status
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
-- 用户表
left join db_digua_business.t_member_user tmu on tmu.id=om.user_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 订单物流表
left join db_digua_business.t_order_logistics tol on tol.order_id=om.id
-- 资方订单分期表
left join (select order_id, refund_date, reality_refund_date from db_rent.ya_merchant_order_stages where sort=2) ymos on ymos.order_id=om.id
where date_format(om.create_time, '%Y-%m-%d')>='2025-05-01'
and date_format(om.create_time, '%Y-%m-%d')<='2025-05-31'
'''
df = query(sql)
df

,create_time,order_number,order_id,客户姓名,delivery_province_name,delivery_city_name,channel_name,refund_date,reality_refund_date,id_card_num,status
0,2025-05-01 00:09:06,A2025050100090637,2531737,朱静依,江苏省,苏州市,顶部搜索框的搜索结果页,2025-06-04,2025-05-31 03:31:16,320584199906163528,4
1,2025-05-01 01:41:07,A20250501014106122,2532037,张玉琦,江苏省,泰州市,顶部搜索框的搜索结果页,2025-06-04,2025-05-26 17:45:07,321202200501275412,4
2,2025-05-01 03:06:21,A2025050103062017,2532211,王东,湖北省,恩施土家族苗族自治州,芝麻信用,2025-06-04,2025-05-30 16:15:50,422826199311015516,4
3,2025-05-01 04:37:05,A2025050104370440,2532341,代高原,内蒙古自治区,巴彦淖尔市,顶部搜索框的搜索结果页,2025-06-04,2025-06-05 11:31:02,152801199811171811,4
4,2025-05-01 08:52:37,A20250501085237110,2532817,刘兰婷,陕西省,西安市,顶部搜索框的搜索结果页,2025-06-04,2025-05-24 17:01:19,622722200502102024,4
...,...,...,...,...,...,...,...,...,...,...,...
4826,2025-05-31 21:32:21,A2025053121322185,2781977,常远方,河南省,郑州市,芝麻信用,2025-07-03,2025-06-11 22:28:53,410182198907050317,4
4827,2025-05-31 22:13:31,A2025053122133047,2782087,肖宇豪,湖南省,长沙市,芝麻信用,2025-07-03,2025-07-02 16:30:03,431381200501300098,4
4828,2025-05-31 23:03:12,A2025053123031215,2782242,徐吴坤,江苏省,南通市,芝麻信用,2025-07-03,2025-07-03 18:24:29,320684199601026712,4
4829,2025-05-31 23:30:08,A20250531233008105,2782331,柯锐杰,广东省,潮州市,芝麻信用,2025-07-03,2025-06-29 11:43:45,445121198901222816,4


In [12]:
t_date = datetime.now().date()
df.loc[:, 'now_day'] = pd.to_datetime(datetime.now().date())
df.loc[:, '下单日期'] = pd.to_datetime(df.create_time.dt.date)
df.loc[:, 'reality_refund_date'] = df.reality_refund_date.dt.date
# 首逾：sort=2，refund_date<now_day|reality_refund_date>refund_date
df.loc[:, 'overdue_day'] = np.where(df.reality_refund_date.isna(), np.where(df.refund_date<df.now_day, df.now_day-df.refund_date, 0), 
                                    np.where(df.reality_refund_date>df.refund_date, df.reality_refund_date-df.refund_date, 0))
df.loc[:, 'overdue_day'] = df.overdue_day.apply(lambda x: x.days)
df


,create_time,order_number,order_id,客户姓名,delivery_province_name,delivery_city_name,channel_name,refund_date,reality_refund_date,id_card_num,status,now_day,下单日期,overdue_day
0,2025-05-01 00:09:06,A2025050100090637,2531737,朱静依,江苏省,苏州市,顶部搜索框的搜索结果页,2025-06-04,2025-05-31,320584199906163528,4,2025-07-16,2025-05-01,0
1,2025-05-01 01:41:07,A20250501014106122,2532037,张玉琦,江苏省,泰州市,顶部搜索框的搜索结果页,2025-06-04,2025-05-26,321202200501275412,4,2025-07-16,2025-05-01,0
2,2025-05-01 03:06:21,A2025050103062017,2532211,王东,湖北省,恩施土家族苗族自治州,芝麻信用,2025-06-04,2025-05-30,422826199311015516,4,2025-07-16,2025-05-01,0
3,2025-05-01 04:37:05,A2025050104370440,2532341,代高原,内蒙古自治区,巴彦淖尔市,顶部搜索框的搜索结果页,2025-06-04,2025-06-05,152801199811171811,4,2025-07-16,2025-05-01,1
4,2025-05-01 08:52:37,A20250501085237110,2532817,刘兰婷,陕西省,西安市,顶部搜索框的搜索结果页,2025-06-04,2025-05-24,622722200502102024,4,2025-07-16,2025-05-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4826,2025-05-31 21:32:21,A2025053121322185,2781977,常远方,河南省,郑州市,芝麻信用,2025-07-03,2025-06-11,410182198907050317,4,2025-07-16,2025-05-31,0
4827,2025-05-31 22:13:31,A2025053122133047,2782087,肖宇豪,湖南省,长沙市,芝麻信用,2025-07-03,2025-07-02,431381200501300098,4,2025-07-16,2025-05-31,0
4828,2025-05-31 23:03:12,A2025053123031215,2782242,徐吴坤,江苏省,南通市,芝麻信用,2025-07-03,2025-07-03,320684199601026712,4,2025-07-16,2025-05-31,0
4829,2025-05-31 23:30:08,A20250531233008105,2782331,柯锐杰,广东省,潮州市,芝麻信用,2025-07-03,2025-06-29,445121198901222816,4,2025-07-16,2025-05-31,0


In [13]:
df_new = df[(df.overdue_day>0)&(df.reality_refund_date.isna())&(df.status==4)]
# 获取需要字段
df_new.loc[:, '用户来源'] = df_new.channel_name.fillna('未知来源')
df_new.loc[:, '省/市'] = df_new.delivery_province_name+df_new.delivery_city_name
df_new.loc[:, '年龄'] = df_new.apply(lambda x: clean.get_age(x['id_card_num'],x['下单日期']), axis=1)
df_new.loc[:, '性别'] = df_new.apply(lambda x: clean.get_gender(x.id_card_num), axis=1)
df_new

,create_time,order_number,order_id,客户姓名,delivery_province_name,delivery_city_name,channel_name,refund_date,reality_refund_date,id_card_num,status,now_day,下单日期,overdue_day,用户来源,省/市,年龄,性别
8,2025-05-01 10:58:38,A20250501105837366,2533527,雷雨欣,河南省,周口市,芝麻信用,2025-06-04,NaT,411626200603076160,4,2025-07-16,2025-05-01,42,芝麻信用,河南省周口市,19,女
11,2025-05-01 11:05:17,A2025050111051738,2533576,吴其烈,辽宁省,沈阳市,顶部搜索框的搜索结果页,2025-06-04,NaT,210321200205110013,4,2025-07-16,2025-05-01,42,顶部搜索框的搜索结果页,辽宁省沈阳市,22,男
32,2025-05-01 13:35:41,A20250501133541240,2534717,郑晶晶,广东省,清远市,搜索下挂-商品,2025-06-04,NaT,441881199306271440,4,2025-07-16,2025-05-01,42,搜索下挂-商品,广东省清远市,31,女
60,2025-05-01 16:17:05,A20250501161705160,2536091,张怡蓉,浙江省,湖州市,搜索-租手机,2025-06-04,NaT,330501200403098521,4,2025-07-16,2025-05-01,42,搜索-租手机,浙江省湖州市,21,女
63,2025-05-01 16:26:19,A20250501162619236,2536167,曹洪钟,山西省,临汾市,搜索下挂-商品,2025-06-04,NaT,141024200611090375,4,2025-07-16,2025-05-01,42,搜索下挂-商品,山西省临汾市,18,男
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4749,2025-05-30 20:13:34,A2025053020133470,2776126,李佳斌,内蒙古自治区,乌兰察布市,芝麻信用,2025-07-02,NaT,152628198704110233,4,2025-07-16,2025-05-30,14,芝麻信用,内蒙古自治区乌兰察布市,38,男
4769,2025-05-31 11:15:11,A20250531111511136,2778587,仇代雪,江苏省,盐城市,搜索下挂-商品,2025-07-03,NaT,320928198810172223,4,2025-07-16,2025-05-31,13,搜索下挂-商品,江苏省盐城市,36,女
4785,2025-05-31 14:05:34,A2025053114053336,2779678,洛桑曲珍,湖南省,长沙市,芝麻信用,2025-07-03,NaT,542224200606070026,4,2025-07-16,2025-05-31,13,芝麻信用,湖南省长沙市,18,女
4799,2025-05-31 15:30:06,A20250531153005201,2780248,曾宪隆,广东省,惠州市,芝麻信用,2025-07-03,NaT,350821199810244717,4,2025-07-16,2025-05-31,13,芝麻信用,广东省惠州市,26,男


In [14]:
sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
df_name = query(sql_name)
df_name


,order_id,分配人,update_time
0,703280,李巧凤,2023-10-16 16:02:04
1,703287,小谢,2023-10-16 16:02:04
2,703288,小张,2023-10-16 16:02:04
3,699105,李巧凤,2023-10-16 16:09:36
4,699143,李巧凤,2023-10-16 16:09:36
...,...,...,...
218106,3259986,黄兰娟,2025-07-16 17:55:41
218107,3262495,何静,2025-07-16 17:56:11
218108,3262495,黄兰娟,2025-07-16 17:56:12
218109,3261859,廖丽敏,2025-07-16 17:56:30


In [15]:
df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
# 创建分配人映射关系 
name_dict = {
        '小张': '李巧玲',
        '小周': '李巧凤',
        '小南': '刘三妹',
        '小何': '何静',
        '小谢': '谢金凤',
        '小咪': '杨健',
        '小慧': '林思慧',
        '小滢': '胡彩滢',
        '小星': '廖丽敏',
        '小兰': '黄兰娟',
        '小晚': '周莹',
        '小芳': '罗芳'
        }
name_list = ['李巧玲','李巧凤','刘三妹','何静','谢金凤','廖丽敏','李楠','黄兰娟','杨健','林思慧','胡彩滢','周莹','罗芳', '周念慈', '周汉鸿']
# 匹配映射字典
df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
# df_name = df_name[df_name.信审员.isin(name_list)]
df_name

,order_id,分配人,update_time,信审员
218110,3262681,何静,2025-07-16 17:57:14,何静
218109,3261859,廖丽敏,2025-07-16 17:56:30,廖丽敏
218108,3262495,黄兰娟,2025-07-16 17:56:12,黄兰娟
218106,3259986,黄兰娟,2025-07-16 17:55:41,黄兰娟
218105,3261577,梦飞,2025-07-16 17:54:16,梦飞
...,...,...,...,...
37,702113,李巧凤,2023-10-16 16:09:36,李巧凤
36,702045,李巧凤,2023-10-16 16:09:36,李巧凤
2,703288,小张,2023-10-16 16:02:04,李巧玲
1,703287,小谢,2023-10-16 16:02:04,谢金凤


In [16]:
# 关联分配人
df_all = df_new.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
# 获取台账数据
f_path_ck = "D:/excel_data/台账/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025")
df_all = df_all.merge(df_ck[['订单号', '型号']],left_on="order_number", right_on="订单号", how='left')
df_all.drop_duplicates(subset=["订单号"],inplace=True)
df_all.loc[:, '手机型号'] = df_all.型号.apply(lambda x: str(x).strip(r'无锁|有锁|带锁|95新|准新|99新'))
df_all_new = df_all[['信审员', 'order_number', '客户姓名', '手机型号', '省/市', '用户来源', '性别', '年龄']].rename(columns={'order_number': '订单编号'})
# df_all_new = df_all_new[df_all_new.信审员.isin(name_list)]
# df_all_new.to_excel(f'F:/需求/总需求/5月份首逾_{t_date}.xlsx', index=False)
df_all_new

,信审员,订单编号,客户姓名,手机型号,省/市,用户来源,性别,年龄
0,林思慧,A20250501105837366,雷雨欣,iPhone16ProMax,河南省周口市,芝麻信用,女,19
1,何静,A2025050111051738,吴其烈,iPhone16ProMax,辽宁省沈阳市,顶部搜索框的搜索结果页,男,22
2,林婉婷,A20250501133541240,郑晶晶,iPhone16ProMax,广东省清远市,搜索下挂-商品,女,31
3,黄兰娟,A20250501161705160,张怡蓉,iPhone16ProMax,浙江省湖州市,搜索-租手机,女,21
4,钟桂洪,A20250501162619236,曹洪钟,iPhone16ProMax,山西省临汾市,搜索下挂-商品,男,18
...,...,...,...,...,...,...,...,...
156,周莹,A2025053020133470,李佳斌,iPhone16ProMax,内蒙古自治区乌兰察布市,芝麻信用,男,38
157,蔡斯静,A20250531111511136,仇代雪,iPhone16ProMax,江苏省盐城市,搜索下挂-商品,女,36
158,黄兰娟,A2025053114053336,洛桑曲珍,iPhone16ProMax,湖南省长沙市,芝麻信用,女,18
159,刘三妹,A20250531153005201,曾宪隆,iPhone16ProMax,广东省惠州市,芝麻信用,男,26
